In [41]:
# -*- coding: utf-8 -*-
import sys
import os
import json
from datetime import datetime
import numpy as np
import pandas as pd
import pymysql
import statsmodels.api as sm
import statsmodels.formula.api as smf
import predictions as pr
import finance as fr
import cost

params={'startdate':'2018-10-10','year':25,'size':99.5,'weight':1.5,'averagetime':3.4,
        "scene":"lm_model","type":"month","plant":"solar",
        "construction":150000000,"investment":180000000,"othercost":25000000,
        "principal":120000000,"interest":0.05, "unredeemed":12321321123,"duration":12,
        "repayment_method":"cpm","repayment_term":"m","interest_repayment_term":"q",
        "finance_startdate":"2018-06-13","price_index":0.01,"solar_index":-0.08}

#물가 예상 
price_index=pr.index_predict(params["startdate"],params["year"],params["price_index"]);price_index.name="price_index";
solar_index=pr.index_predict(params["startdate"],params["year"],params["solar_index"]);solar_index.name="solar_index";

#수입예상 및 전체적인 지표구성
revenue=pr.predict(params["scene"],params["startdate"],params["year"])
revenue["rec_price"]=100;
revenue["days"]=revenue.index.day
start=pd.to_datetime(params["startdate"], format='%Y-%m-%d', errors='ignore')#초기 기간 설정
startday=start.date().timetuple()
revenue.days[0]=revenue.days[0]-startday.tm_mday
revenue.days[revenue.shape[0]-1]=startday.tm_mday

revenue=pd.concat([revenue,price_index,solar_index],axis=1)
revenue["generation"]=revenue['days']*params['size']*params['averagetime']*revenue["solar_index"]
revenue['smp_revenue']=revenue['smp_price']*revenue['generation']
revenue['rec_revenue']=revenue['rec_price']*revenue['generation']*params['weight']
revenue["smp_revenue"]=revenue["smp_revenue"].astype(int); revenue["rec_revenue"]=revenue["rec_revenue"].astype(int)

#지출예상
OM_cost=cost.OM_calc(params["startdate"],params["size"],params["year"])
monitoring_cost=cost.monitoring_cost_calc(params["startdate"],params["size"],params["year"])
elec_safety_cost=cost.elec_safety_calc(params["startdate"],params["size"],params["year"])
office_cost=cost.office_cost_calc(params["startdate"],params["size"],params["year"])
other_cost=cost.other_cost_calc(params["startdate"],params["size"],params["year"])
total_cost=pd.concat([OM_cost,monitoring_cost,elec_safety_cost,office_cost,other_cost],axis=1)
cost=pd.concat([total_cost,price_index],axis=1)

cost["OM_cost"]=cost["OM_cost"]*cost["price_index"]
cost["monitoring_cost"]=cost["monitoring_cost"]*cost["price_index"]
cost["elec_safety_cost"]=cost["elec_safety_cost"]*cost["price_index"]
cost["office_cost"]=cost["office_cost"]*cost["price_index"]
cost["other_cost"]=cost["other_cost"]*cost["price_index"]
cost["depreciation"]=params["construction"]/(params["duration"]*12)

#금융구조 예상
if(params["repayment_method"]=="cam"):
    amortization=fr.CAM_calc(params["finance_startdate"],params["duration"],params["principal"],params["interest"],params["repayment_term"],params["interest_repayment_term"])
elif(params["repayment_method"]=="cpm"):
    amortization=fr.CPM_calc(params["finance_startdate"],params["duration"],params["principal"],params["interest"],params["repayment_term"])
amortization.index=amortization.index.shift(n=1,freq="m")
result=pd.concat([revenue,cost,amortization],axis=1)

#전체적인 구조 도출 
result=result.loc[:,['smp_revenue','rec_revenue','OM_cost','monitoring_cost','elec_safety_cost','office_cost','other_cost','depreciation','interest','principal']]
result=result.fillna(0)
def money_trim(array):
    result=round(array,-1)
    result=result.astype(int)
    return result
result=result.apply(money_trim)

#쿼터로 만들기
result_quarter=result.copy()
result_quarter.index=result_quarter.index.to_period("q")
result_quarter=result_quarter.groupby(result_quarter.index).sum()

#연단위 로 만들기
result_year=result.copy()
result_year.index=result_year.index.to_period("y")
result_year=result_year.groupby(result_year.index).sum()

#손익계산서 만들기 
def makeIncomeState(income_state):
    income_state["gross_income"]=income_state["smp_revenue"]+income_state["rec_revenue"]
    income_state["operating_expense"]=income_state["monitoring_cost"]+income_state["elec_safety_cost"]+income_state["office_cost"]+income_state["other_cost"]+income_state["depreciation"]                                                                      
    income_state["operating_income"]=income_state["gross_income"]-income_state["operating_expense"]
    income_state["pretax_net_income"]=income_state["operating_income"]-income_state["interest"]
    def tax(x):
        if x>0:
            return x*0.1
        else:
            return 0
    income_state["tax"]=income_state.pretax_net_income.apply(tax)
    income_state["net_income"]=income_state["pretax_net_income"]-income_state["tax"]
    income_state=income_state.loc[:,["smp_revenue","rec_revenue","gross_income","monitoring_cost","elec_safety_cost","office_cost","other_cost","depreciation","operating_expense","operating_income","interest","pretax_net_income","tax","net_income"]]
    return income_state

income_month=makeIncomeState(result)
income_quarter=makeIncomeState(result_quarter)
income_year=makeIncomeState(result_year)

/Users/youngji/.local/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
